### Setup

Make sure you have the files available from previous demos.

In [0]:
# This cell sets all the configuration parameters to connect to Azure Data Lake
spark.conf.set("fs.azure.account.auth.type.<account_name>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<account_name>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<account_name>.dfs.core.windows.net", "****************************")
spark.conf.set("fs.azure.account.oauth2.client.secret.<account_name>.dfs.core.windows.net", "*******************************")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<account_name>.dfs.core.windows.net", "https://login.microsoftonline.com/************************/oauth2/token")

Verify that cloud storage is accessible

In [0]:
dbutils.fs.ls("abfss://pyspark@warnerdatalake.dfs.core.windows.net/")

Let's load the transactions data with some optimizations

In [0]:
from pyspark.sql import functions as F

# Define the path to the transactions dataset
parquet_path = "abfss://pyspark@warnerdatalake.dfs.core.windows.net//imports//transactions_data.parquet"

# Apply column pruning and predicate pushdown
df_optimized = (
    spark.read.parquet(parquet_path)
        .select("category", "amount")  # Column pruning
        .filter(F.col("amount") > 50)  # Predicate pushdown
        .groupBy("category")
        .agg(F.sum("amount").alias("total_amount"))
)

# Display the optimized DataFrame
df_optimized.limit(5).display()


category,total_amount
Food,3768630.66
Sports,3746511.53
Electronics,3773395.39
Books,3739182.06
Furniture,3745521.08


Verify the predicate pushdown in the plan

In [0]:
#Look for PushedFilters: [GreaterThan(amount,50)] in the output, which confirms predicate pushdown is happening.
df_optimized.explain(True)

== Parsed Logical Plan ==
'Aggregate ['category], ['category, 'sum('amount) AS total_amount#60]
+- Filter (amount#48 > cast(cast(50 as decimal(2,0)) as decimal(10,2)))
   +- Project [category#49, amount#48]
      +- Relation [transaction_id#45L,customer_id#46,transaction_date#47,amount#48,category#49] parquet

== Analyzed Logical Plan ==
category: string, total_amount: decimal(20,2)
Aggregate [category#49], [category#49, sum(amount#48) AS total_amount#60]
+- Filter (amount#48 > cast(cast(50 as decimal(2,0)) as decimal(10,2)))
   +- Project [category#49, amount#48]
      +- Relation [transaction_id#45L,customer_id#46,transaction_date#47,amount#48,category#49] parquet

== Optimized Logical Plan ==
Aggregate [category#49], [category#49, sum(amount#48) AS total_amount#60]
+- Project [category#49, amount#48]
   +- Filter (isnotnull(amount#48) AND (amount#48 > 50.00))
      +- Relation [transaction_id#45L,customer_id#46,transaction_date#47,amount#48,category#49] parquet

== Physical Plan ==


We can optimize retrieval and fit a query pattern with partitioning

In [0]:
# First write the dataframe as a partitioned set of folders
partitioned_path = "abfss://pyspark@warnerdatalake.dfs.core.windows.net//exports//transactions_partitioned"
df_transactions = spark.read.parquet(parquet_path)
df_transactions.write.mode("overwrite").partitionBy("category", "transaction_date").parquet(partitioned_path)


Then we can read it back with the right filter

In [0]:
df_partitioned = (
    spark.read.parquet(partitioned_path)
        .filter(F.col("category") == "Electronics")  # Partition pruning
)

df_partitioned.limit(5).display()


transaction_id,customer_id,amount,category,transaction_date
139,9976,19.82,Electronics,2025-01-03
1445,1895,9.69,Electronics,2025-01-03
2828,8191,79.55,Electronics,2025-01-03
4393,236,94.39,Electronics,2025-01-03
4470,5998,79.38,Electronics,2025-01-03


Verify the partition pruning in the plan

In [0]:
# Look for PartitionFilters: [isnotnull(category), (category = Electronics)], which confirms partition pruning.
df_partitioned.explain(True)

== Parsed Logical Plan ==
'Filter '`=`('category, Electronics)
+- Relation [transaction_id#103L,customer_id#104,amount#105,category#106,transaction_date#107] parquet

== Analyzed Logical Plan ==
transaction_id: bigint, customer_id: int, amount: decimal(10,2), category: string, transaction_date: date
Filter (category#106 = Electronics)
+- Relation [transaction_id#103L,customer_id#104,amount#105,category#106,transaction_date#107] parquet

== Optimized Logical Plan ==
Filter (isnotnull(category#106) AND (category#106 = Electronics))
+- Relation [transaction_id#103L,customer_id#104,amount#105,category#106,transaction_date#107] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [transaction_id#103L,customer_id#104,amount#105,category#106,transaction_date#107] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[abfss://pyspark@warnerdatalake.dfs.core.windows.net/exports/transactio..., PartitionFilters: [isnotnull(category#106), (category#106

Some file formats have even more optimizations like delta

In [0]:
delta_path = "abfss://pyspark@warnerdatalake.dfs.core.windows.net//exports//transactions_delta"

df_transactions.write.format("delta").mode("overwrite").partitionBy("category", "transaction_date").save(delta_path)


Delta supports Z-Ordering, which improves range-based queries (e.g., amount > X)

In [0]:
from delta.tables import DeltaTable

# Load Delta table
delta_table = DeltaTable.forPath(spark, delta_path)

# Optimize Delta Table Storage
delta_table.optimize().executeZOrderBy("amount")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

To confirm all optimizations, let's run .explain() on the Delta table.

In [0]:
df_delta = (
    spark.read.format("delta").load(delta_path)
        .filter((F.col("amount") > 50) & (F.col("category") == "Electronics"))
)

df_delta.explain(True)

# Expected Execution Plan Output
# PushedFilters: [GreaterThan(amount,50)] → ✅ Predicate Pushdown confirmed.
# PartitionFilters: [category=Electronics] → ✅ Partition Pruning confirmed.
# Z-Ordering by amount → ✅ File skipping optimization confirmed.


== Parsed Logical Plan ==
'Filter 'and('`>`('amount, 50), '`=`('category, Electronics))
+- Relation [transaction_id#774L,customer_id#775,transaction_date#776,amount#777,category#778] parquet

== Analyzed Logical Plan ==
transaction_id: bigint, customer_id: int, transaction_date: date, amount: decimal(10,2), category: string
Filter ((amount#777 > cast(cast(50 as decimal(2,0)) as decimal(10,2))) AND (category#778 = Electronics))
+- Relation [transaction_id#774L,customer_id#775,transaction_date#776,amount#777,category#778] parquet

== Optimized Logical Plan ==
Filter ((isnotnull(amount#777) AND isnotnull(category#778)) AND ((amount#777 > 50.00) AND (category#778 = Electronics)))
+- Relation [transaction_id#774L,customer_id#775,transaction_date#776,amount#777,category#778] parquet

== Physical Plan ==
*(1) Project [transaction_id#774L, customer_id#775, transaction_date#776, amount#777, category#778]
+- *(1) Filter ((if (isnotnull(_databricks_internal_edge_computed_column_skip_row#928)) (_d